In [ ]:
using PlotlyJS
using Unitful
using PhysicalConstants.CODATA2018: ε_0, c_0, h

using RefractiveIndexDatabase
using Permittivity

In [ ]:
for material in RefractiveIndexDatabase.search(shelf="main", book="Au")
    println(material)
end

In [ ]:
Au = get_material("main", "Au", "Babar")

In [ ]:
# Convert into permititvity

λ = Au.λ
m = Au.n + Au.k * im
ε = m.^2;

In [ ]:
@show (h * c_0) / u"nm" |> u"eV"

ω = ustrip.(h* c_0 ./ (λ * 1u"μm") .|> u"eV")
trace1 = scatter(x=ω, y=real(ε), name="Re(ε)", mode="markers")
trace2 = scatter(x=ω, y=imag(ε), name="Im(ε)", mode="markers")

# layout = Layout(;title="", xaxis=attr(title="Wavelength [μm]", type="log"), yaxis=attr(title="ε_r", type="log"))
layout1 = Layout(;xaxis=attr(title="[eV]", range=[1,5]), yaxis=attr(title="ε_r", range=[-70,10]))
layout2 = Layout(;xaxis=attr(title="[eV]", range=[1,5]), yaxis=attr(title="ε_r", range=[0,7]))

[plot(trace1, layout1) plot(trace2, layout2)]


In [ ]:
# bug
# u"nm" |> u"μm"

In [ ]:
@show (h * c_0) / u"800.0nm" |> u"eV"
@show (h * c_0) / u"600.0nm" |> u"eV"

# DOI: 10.1002/adom.201300457
ε_inf = 2.2715 
ω_p = 8.9234 * u"eV"
Γ_p = 0.042389 * u"eV"

model = DrudeModel(ε_inf, ustrip(ω_p), ustrip(Γ_p))
ε_drude = model.(ω)

trace1_drude = scatter(x=ω, y=real(ε_drude), name="Drude Re(ε(ω))")
trace2_drude = scatter(x=ω, y=imag(ε_drude), name="Drude Im(ε(ω))")
layout = Layout(;title="Interpolated values", xaxis=attr(title="Frequency [THz]"), yaxis=attr(title="ε"))

[plot([trace1, trace1_drude], layout1) plot([trace2,trace2_drude], layout2)]

In [ ]:
# Frequency 

ω = ustrip.(2π * c_0 ./ (λ * 1u"μm") .|> u"THz")

trace1 = scatter(x=ω/2π, y=real(m), name="Re(m)", mode="markers")
trace2 = scatter(x=ω/2π, y=imag(m), name="Im(m)", mode="markers")
# trace1 = scatter(x=ω/2π, y=real(ε), name="Re(ε)", mode="markers")
# trace2 = scatter(x=ω/2π, y=imag(ε), name="Im(ε)", mode="markers")

layout = Layout(;title="", xaxis=attr(title="Frequency [THz]", type="log"), yaxis=attr(title="ε_r", type="log"))
# layout = Layout(;title="", xaxis=attr(title="Frequency [THz]", type="log"), yaxis=attr(title="ε_r"))

plot([trace1, trace2], layout)


In [ ]:
# interpolation

int = PermittivityTable(ω, ε)

ω_int = LinRange(extrema(ω)..., 1000)
ε_int = int.(ω)

trace1 = scatter(x=ω/2π, y=real(ε_int), name="Re(ε(ω))")
trace2 = scatter(x=ω/2π, y=imag(ε_int), name="Im(ε(ω))")
trace3 = scatter(x=ω/2π, y=real(ε), name="Re(ε)", mode="markers")
trace4 = scatter(x=ω/2π, y=imag(ε), name="Im(ε)", mode="markers")

layout = Layout(;title="Interpolated values", xaxis=attr(title="Frequency [THz]"), yaxis=attr(title="ε_r"))

plot([trace1, trace2, trace3, trace4], layout)


In [ ]:
# range of interest

inds = 0.6 .< λ .< 0.750
@show sum(inds)

trace1 = scatter(x=λ[inds], y=real(m[inds]), name="Re(m)", mode="markers+lines")
trace2 = scatter(x=λ[inds], y=imag(m[inds]), name="Re(m)", mode="markers+lines")

trace1 = scatter(x=λ[inds], y=real(ε[inds]), name="Re(ε)", mode="markers+lines")
trace2 = scatter(x=λ[inds], y=imag(ε[inds]), name="Re(ε)", mode="markers+lines")

# layout = Layout(;title="", xaxis=attr(title="Wavelenght [um]", type="log"), yaxis=attr(title="ε_r", type="log"))
layout1 = Layout(xaxis=attr(title="Wavelenght [um]", type="log"), yaxis=attr(title="ε_r"))
layout2 = Layout(xaxis=attr(title="Wavelenght [um]", type="log"), yaxis=attr(title="ε_r"))

[plot(trace1, layout1)  plot(trace2, layout2)]

In [ ]:
# All in one together


using Unitful
using PhysicalConstants.CODATA2018: ε_0, c_0, h

using RefractiveIndexDatabase
using Permittivity

Au = get_material("main", "Au", "Babar")

ω = ustrip(h * c_0 / u"μm" |> u"eV") ./ Au.λ
ε = @. (Au.n^2 - Au.k^2) + (2 * Au.n * Au.k) * im

int = PermittivityTable(ω, ε)

λ_range = (632.0:1.0:732.0)u"nm"
ω_range = ustrip(@. (h * c_0) / λ_range  |> u"eV" )

ε_int = int.(ω_range)

trace1 = scatter(x=ω_range, y=real(ε_int), name="Re(ε(ω))")
trace2 = scatter(x=ω_range, y=imag(ε_int), name="Im(ε(ω))")
trace3 = scatter(x=ω, y=real(ε), name="Re(ε)", mode="markers")
trace4 = scatter(x=ω, y=imag(ε), name="Im(ε)", mode="markers")

layout1 = Layout(xaxis=attr(title="[eV]", range=extrema(ω_range)), yaxis=attr(title="ε", range=[-20,0]))
layout2 = Layout(xaxis=attr(title="[eV]", range=extrema(ω_range)), yaxis=attr(title="ε", range=[0,1]))

[plot([trace1, trace3], layout1) plot([trace2, trace4], layout2)]


In [ ]:
# All in one together


using Unitful
using PhysicalConstants.CODATA2018: ε_0, c_0, h

using RefractiveIndexDatabase
using Permittivity

Au = get_material("main", "Au", "Babar")

λ = Au.λ * 1000  # u"nm"
ε = @. (Au.n^2 - Au.k^2) + (2 * Au.n * Au.k) * im

int = PermittivityTable(λ, ε)

λ_range = 632.0:1.0:732.0   # u"nm"

ε_int = int.(λ_range)

trace1 = scatter(x=λ_range, y=real(ε_int), name="Re(ε(ω))")
trace2 = scatter(x=λ_range, y=imag(ε_int), name="Im(ε(ω))")
trace3 = scatter(x=λ, y=real(ε), name="Re(ε)", mode="markers")
trace4 = scatter(x=λ, y=imag(ε), name="Im(ε)", mode="markers")

layout1 = Layout(xaxis=attr(title="[nm]", range=extrema(λ_range)), yaxis=attr(title="ε", range=[-20,0]))
layout2 = Layout(xaxis=attr(title="[nm]", range=extrema(λ_range)), yaxis=attr(title="ε", range=[0,1]))

[plot([trace1, trace3], layout1) plot([trace2, trace4], layout2)]
